## Workshop om Ragas

#### Kilder:
- [RAGAS](https://docs.ragas.io/en/stable/) <img src="ragas.png" alt="Ragas logo" width="25"/>

## Begrepsforklaringer
##### RAG
RAG står for Retrieval-Augmented Generation og gir generativ kunstig intelligens modeler informasjonsinnhenting muligheter. Dette betyr RAG-en hjelper generative KI ved å gi den tilgang til relevant informasjon som kan hjelpe med å svare på spørsmål fra bruker. 
Informasjonen blir på forhånd lagret omgjort til LLM-embeddings (store vektorer) og lagret i vektor-databaser. Før den generative KI-en svarer, blir brukerspørsmålet gjort om til en egen LLM-embedding og den embeddingen blir brukt til å sammenligne med de andre vektorene i vektor-databasen. Her blir ofte "Cosine-similarity" brukt for å finne vektorene som ligner mest. Deretter blir informasjonen som er mest relevant til spørsmålet, gitt til KI-en.

### Hva er ragas?
Ragas er et bra dokumentert (!), open-source bibliotek som lar deg evaluere LLM-applikasjoner og RAG-er. Viktig for denne evalueringen er metrics.

<img src="metrics_mindmap.png" alt="Metrics Mindmap" width="500"/>

To typer metrics:

##### LLM Based
- Bruker LLM til å vurdere. 
- Non-deterministisk ved at LLM ikke alltid vil returnere det samme resultatet
- Likevel vist seg å være mer nøyaktige og nærmere menneskelig evaluering

##### Non-LLM Based
- Bruker **ikke** LLM til å vurdere
- Deterministiske 
- Bruker tradisjonelle metoder for å evaluere
- Mindre nøyaktige sammenlignet med menneskelig evaluering


To andre kategorier:
##### Single Turn Metrics
- Evaluerer basert på én runde med interaksjon mellom bruker og generativ KI

##### Multiple Turn Metrics
- Evalierer basert på flere runder med interaksjon mellom bruker og generativ KI


### Hvordan evaluere med Ragas?
For å evaluere hvor god en generativ KI trenger man 3 ting:
- Spørsmål
- Svar
- Referanse/riktig svar

Med dette kan man evaluere om svaret KI-en gir stemmer opp mot svaret vi forventer.

For å evaluere hvor god en RAG er til å gi riktig informasjon til KI-en trenger man 4 ting:
- Spørsmål
- Svar
- Referanse/riktig svar
- Gitt kontekst/informasjon

Den siste (gitt kontekst) er viktig når man skal vurdere hvor svikten i system ligger. Hvis det er gitt feil kontekst, er det RAG-en som har mislyktes. Hvis det er riktig kontekst, men feil svar, er det KI-en som har mislyktes. 

Under skal vi se hvordan vi kan evaluere en RAG.

In [ ]:
%pip install -U ragas langchain langchain_openai faiss-cpu pandas

### Sette env-variabler

In [2]:
import os

os.environ["LANGCHAIN_API_KEY"] = "your_key"
os.environ["LANGCHAIN_TRACING_V2"] = "false"
os.environ["LANGCHAIN_PROJECT"] = "Ragas Tutorial"

os.environ["OPENAI_API_KEY"] = "your_key"

### Lese fil og lage vektor-database

In [3]:
from ragas import EvaluationDataset
from langchain_openai import OpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter, MarkdownHeaderTextSplitter, MarkdownTextSplitter
from langchain_experimental.text_splitter import SemanticChunker
from langchain_core.documents import Document
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper
from ragas.metrics import LLMContextRecall, ContextEntityRecall, ContextPrecision, FactualCorrectness
from langchain_community.vectorstores import FAISS
from ragas.utils import safe_nanmean
from langchain_openai import ChatOpenAI
import pandas as ps

# Setter opp RAG-en

# Leser dokumentene vi vil legge inn i RAG-en
filepath = './dnd_doc.md'


dnd_document: str = ""
with open(filepath, encoding="utf-8") as f:
    dnd_document = f.read()

# Gjør teksten om til en Document-klasse fra Langchain
dnd_document = [Document(dnd_document)]

# Splitter dokumentet med en tekstsplitter fra Langchain
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0) # TextSplitter
splits = text_splitter.split_documents(dnd_document)

# Lager en vektor-database retriever
k = 2
score_threshold = 0.7
vector_retriever = FAISS.from_documents(splits, OpenAIEmbeddings()).as_retriever(
            search_type="similarity_score_threshold",
            search_kwargs = {
                "k": k,
                "score_threshold": score_threshold
                }
        )

### Generere svar til spørsmålene i CSV-filen

In [4]:
llm = ChatOpenAI(
            model= "gpt-4o-mini",
            temperature=0,
            max_tokens=16384
        )

def convert_docs_to_strings(docs: list[Document]) -> list[str]:
        """Convert Objects of Document type to an list of strings"""
        return [doc.page_content for doc in docs]

def get_relevant_docs(query: str) -> list[str]:
    return convert_docs_to_strings(vector_retriever.invoke(input=query))

def generate_answer(query: str, relevant_doc: list[Document]):
        """Generate an answer for a given query based on the most relevant document."""
        prompt = f"question: {query}\n\nDocuments: {relevant_doc}"
        messages = [
            ("system", "You are a helpful assistant that answers questions based on given documents only."),
            ("human", prompt),
        ]
        ai_msg = llm.invoke(messages)
        return ai_msg.content

dataset = []

df = ps.read_csv("./sample_questions_dnd.csv")
querys = df["question"].tolist()
responses = df["answer"].tolist()

for query, reference in zip(querys, responses):

    relevant_docs = get_relevant_docs(query=query)
    response = generate_answer(query, relevant_docs)
    # Legger til i datasettet spørsmålet, kontekst, response fra KI og referansen/riktig svar
    dataset.append(
        {
            "user_input":query,
            "retrieved_contexts":relevant_docs,
            "response":response,
            "reference":reference
        }
    )

## Evaluere datasettet
For å evaluere datasettet kan man bruke mange ulike metrics, alle listet på nettsiden til Ragas (https://docs.ragas.io/en/latest/concepts/metrics/available_metrics/)

- Context Precision
- Context Recall
- Context Entities Recall
- Noise Sensitivity
- Response Relevancy
- Faithfulness
- Multimodal Faithfulness
- Multimodal Relevance
- Factual Correctness
- Semantic Similarity
- Non LLM String Similarity
- BLEU Score
- ROUGE Score
- String Presence
- Exact Match

Her kommer vi til å bruke to til å vurdere RAG-en (Context Recall og Context Precision) og en for å vurdere responsen til LLM-en (Factual Correctness)


In [ ]:
# Evaluerer datasettet

evaluation_dataset = EvaluationDataset.from_list(dataset)

evaluator_llm = LangchainLLMWrapper(llm)

result = evaluate(
    dataset=evaluation_dataset,
    metrics=[LLMContextRecall(), ContextPrecision(), FactualCorrectness()],
    llm=evaluator_llm
)

context_recall = safe_nanmean(result["context_recall"])
context_precision = safe_nanmean(result["context_precision"])
factual_correctness = safe_nanmean(result["factual_correctness"])

print(context_recall, context_precision, factual_correctness)

### Teste ulike parametere for å finne beste kombinasjon for RAG-en
Basert på resultatene kan vi forbedre RAG-en ved å endre på ulike aspekter av RAG-en. Det er to måter måter man kan gjøre det på:

#### Splitters

Det første aspektet er hvordan vi deler opp dokumentet før vi legger det inn i databasen. Med klassen RecursiveCharacterSplitter kan vi endre på hvor store chunks vi lager og hvor mye overlap de har. Dette bestemmer hvor mye informasjon det er i hver kontekst vi gir LLM-en. 

Mindre kontekst:
- Mindre informasjon til LLM-en per spørring
- Kan svekke nøyaktigheten noe, 
- Krever mindre tokens. 

Større kontekst:
- Mer informasjon til LLM-en
- Kan øke nøyaktigheten
- Krever mer tokens

Man kan også bruke andre splittere (eller lage egne). Her er andre splittere fra Langchain som kan brukes:
- CharacterTextSplitter
- MarkdownHeaderTextSplitter
- RecursiveJsonSplitter
- SemanticChunker

#### Retrivers (vektordatabase)
Hvordan dataen blir lagret og hvordan den blir hentet er mye å si på nøyaktigheten til RAG-en. Med FAISS som vectorstore er det tre parametere man kan endre på:
- Search Type
   - similarity
   - similiarity_score_threshold
   - mmr
- k
    - Maksimalt antall kontekster som blir hentet

- score_threshold
    - Likhetsscore for at en kontekst skal kunne bli hentet av k

Man kan også bruke andre måter å hente dokumenter på. I tillegg til egen implementerte metoder, har Langchain flere ulike retrievers:
- ParentDocumentRetriever, 
- EnsembleRetriever 
- MultiVectorRetriever
- BM25Retriver



##### Prøv dere fram og forsøk å få høyest mulig score!!



##### Hvis tid
## Optuna
Et bibliotek for å automatisere testing av parametere. Man spesifiserer en oppgave med ulike parametere (som bygging av en RAG) og sier hvilke verdier som den skal prøve å forbedre (for eksempel Context Recall fra RAGAS). Dette kan gjentas så mange ganger man vil og kan pararalliseres. Til slutt kan resultatene fra utprøvingen vises i Optuna sitt eget dashboard.

Se filen **optuna_test.py** for kode

In [ ]:
%pip install -U optuna optuna-dashboard optunahub logging cmaes torch